## Sparse Orthonormal Transforms
---

I implemented the SOT algorithm based on "Approximation and Compression with Sparse Orthonormal Transforms, Osman G. Sezer, Onur G. Guleryuz, and Yucel Altunbasak."

## Get Libraries

In [ ]:
!pip install bitarray
!pip install huffman
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
import pickle
import bitarray
import struct
import math
import heapq
from collections import namedtuple
from collections import Counter
from PIL import Image
import huffman
import ast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 6.4 MB/s eta 0:00:00


## Initialize, Cost Functions and SOT algorithm

In [ ]:
def init_blocks(image, n):
    """Divide the image into non-overlapping blocks of size n x n."""
    h, w = image.shape
    return [image[i:i+n, j:j+n] for i in range(0, h, n) for j in range(0, w, n)]

In [ ]:
def thresholding(coefficients, lambda_value):
    """Apply thresholding to enforce sparsity."""
    threshold = np.sqrt(lambda_value)
    return np.where(np.abs(coefficients) >= threshold, coefficients, 0)

In [ ]:
def init_transform(N):
  """
  Initialize the transform.
  """
  return np.identity(N)

In [ ]:
def get_coefficients(transform, block, lambda_value):
    """Compute sparse coefficients using the current transform. [1]"""
    x_vec = block.flatten()
    coeffs = np.dot(transform.T, x_vec)
    return thresholding(coeffs, lambda_value)

In [ ]:
# Generate a 16x16 grayscale image with random pixel values (0-255) to test the functions
image_16x16 = np.random.randint(0, 256, (16, 16), dtype=np.uint8)
G = init_transform(64)
blocks = init_blocks(image_16x16,8)
coeff_blocks = [get_coefficients(G,block,15625) for block in blocks]
print(image_16x16)
print(G)
print(blocks)
print(coeff_blocks)
print(coeff_blocks[0].shape)

So far so good. All the above functions are working as intended.

In [ ]:
def update_transform(blocks, coefficients):
    """
    Update transform using covariance-based optimization.
    This transform is learned from the entire image but then applied block by block.
    Refer to Proposition 3: optimize G for a given set of coefficients. [2][3][5]
    """
    cov = sum(np.outer(a, b.flatten()) for a, b in zip(coefficients, blocks))
    U, _, Vh = np.linalg.svd(cov, full_matrices=False)
    return np.dot(U, Vh)

In [ ]:
def distortion_cost(block, transform, coefficients):
    """Compute distortion cost ||block - G*coefficients||^2. [4]"""
    reconstructed = np.dot(transform, coefficients)
    return np.linalg.norm(block.flatten() - reconstructed) ** 2

def complexity_cost(lambda_value, coefficients):
    """Compute sparsity cost as lambda * l0_norm."""
    return lambda_value * np.count_nonzero(coefficients)

In [ ]:
def sot(image, n, lambda_, epsilon, max_iters):
    """Train a single transform using the given parameters."""

    G = init_transform(n)

    blocks = init_blocks(image, n) # initialize blocks
    prev_cost = float('inf') # initialize to inf

    for block in blocks:
        coefficients = get_coefficients(G, block, lambda_) # get coeffs
        G = update_transform(block, coefficients) # update G
        cost = distortion_cost(block,G,coefficients) + complexity_cost(lambda_, coefficients) # calculate cost

        if np.abs(prev_cost - cost) < epsilon:  # Check for convergence
            break
        prev_cost = cost # update previous cost

    return G, coefficients

## References

- [1] Python Tutorial https://www.pythontutorial.net/python-numpy/numpy-flatten/
- [2] Numpy Docs https://numpy.org/doc/stable/reference/generated/numpy.outer.html
- [3] Numpy Docs https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
- [4] Numpy Docs https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html
- [5] G4G https://www.geeksforgeeks.org/zip-in-python/
- Adam Colton https://github.com/theAdamColton/spiht/blob/main/spiht/spiht_py.py